In [3]:
from Bio import SeqIO

In [4]:
records = list(SeqIO.parse("dna2.fasta", "fasta"))

In [5]:
len(records)

18

In [76]:
records_dict = {}
for record in records:
    print("%s %i" % (record.id, len(record)))
    records_dict[record.id] = len(record)

gi|142022655|gb|EQ086233.1|91 4635
gi|142022655|gb|EQ086233.1|304 1151
gi|142022655|gb|EQ086233.1|255 4894
gi|142022655|gb|EQ086233.1|45 3511
gi|142022655|gb|EQ086233.1|396 4076
gi|142022655|gb|EQ086233.1|250 2867
gi|142022655|gb|EQ086233.1|322 442
gi|142022655|gb|EQ086233.1|88 890
gi|142022655|gb|EQ086233.1|594 967
gi|142022655|gb|EQ086233.1|293 4338
gi|142022655|gb|EQ086233.1|75 1352
gi|142022655|gb|EQ086233.1|454 4564
gi|142022655|gb|EQ086233.1|16 4804
gi|142022655|gb|EQ086233.1|584 964
gi|142022655|gb|EQ086233.1|4 2095
gi|142022655|gb|EQ086233.1|277 1432
gi|142022655|gb|EQ086233.1|346 115
gi|142022655|gb|EQ086233.1|527 2646


In [77]:
max_len = max(records_dict.values())
print("Max length is %i bp" % max_len)
for key in records_dict.keys():
    if records_dict[key] == max_len:
        print (key)

Max length is 4894 bp
gi|142022655|gb|EQ086233.1|255


In [78]:
min_len = min(records_dict.values())
print("Min length is %i bp" % min_len)
for key in records_dict.keys():
    if records_dict[key] == min_len:
        print (key)

Min length is 115 bp
gi|142022655|gb|EQ086233.1|346


In [6]:
#record = records[0]
#records2 = list(SeqIO.parse("easy.fa", "fasta"))
#record = records2[1]


#record.seq

Seq('CTAGGGCATCTAGAAAGGCAT')

In [83]:
def get_longest(record, frames=[0,1,2]):
    "Returns the longest ORF in a particular sequence"
    orf_dict=  {}
    longest_orf = 0
    longest_start = 0

    #forward frames
    for frame in frames:
        current_loc = frame
        while current_loc<len(record.seq):
            current_triplet = record[current_loc:current_loc+3]
            if current_triplet.seq == 'ATG':
                orf_current = current_loc
                while orf_current<len(record.seq):
                    orf_triplet = record[orf_current:orf_current+3]
                    if orf_triplet.seq in ['TAA', 'TAG', 'TGA']:
                        orf_dict[current_loc] = orf_current
                        break
                    orf_current += 3
            current_loc += 3

    #reverse frames
    for frame in frames:
        current_loc = frame
        while current_loc<len(record.seq):
            current_triplet = record.seq.reverse_complement()[current_loc:current_loc+3]
            if current_triplet == 'ATG':
                orf_current = current_loc
                while orf_current<len(record.seq):
                    orf_triplet = record.seq.reverse_complement()[orf_current:orf_current+3]
                    if orf_triplet in ['TAA', 'TAG', 'TGA']:
                        # because sequence has been reversed, location is actually count from end
                        orf_dict[len(record.seq) - current_loc - 1] = len(record.seq) - orf_current - 1
                        break
                    orf_current += 3
            current_loc += 3

    for key in orf_dict.keys():
        #actual length is end codon - start codon + 3 (to include all bases)
        orf_len = abs(orf_dict[key] - key) + 3
        if orf_len > longest_orf:
            longest_orf = orf_len
            #actual location is location + 1 (as nucleotide counting from 1)
            longest_start = key + 1

    return(longest_start, longest_orf)

In [84]:
def get_longest_for(record, frames=[0,1,2]):
    "Returns the longest ORF in a particular sequence"
    orf_dict=  {}
    longest_orf = 0
    longest_start = 0

    #forward frames
    for frame in frames:
        current_loc = frame
        while current_loc<len(record.seq):
            current_triplet = record[current_loc:current_loc+3]
            if current_triplet.seq == 'ATG':
                orf_current = current_loc
                while orf_current<len(record.seq):
                    orf_triplet = record[orf_current:orf_current+3]
                    if orf_triplet.seq in ['TAA', 'TAG', 'TGA']:
                        orf_dict[current_loc] = orf_current
                        break
                    orf_current += 3
            current_loc += 3

    for key in orf_dict.keys():
        #actual length is end codon - start codon + 3 (to include all bases)
        orf_len = abs(orf_dict[key] - key) + 3
        if orf_len > longest_orf:
            longest_orf = orf_len
            #actual location is location + 1 (as nucleotide counting from 1)
            longest_start = key + 1

    return(longest_start, longest_orf)

In [85]:
overall_longest_start = 0
overall_longest_orf = 0
overall_longest_gene = ""

for record in records:
    longest_start, longest_orf = get_longest_for(record, [1])
    if longest_orf>overall_longest_orf:
        overall_longest_orf = longest_orf
        overall_longest_start = longest_start
        overall_longest_gene = record.id


print ("Longest ORF is in %s starting at %i and is %i bp long." % (overall_longest_gene, 
                                                                   overall_longest_start, 
                                                                   overall_longest_orf))



Longest ORF is in gi|142022655|gb|EQ086233.1|16 starting at 3071 and is 1458 bp long.


In [86]:
overall_longest_start = 0
overall_longest_orf = 0
overall_longest_gene = ""

for record in records:
    longest_start, longest_orf = get_longest_for(record, [2])
    if longest_orf>overall_longest_orf:
        overall_longest_orf = longest_orf
        overall_longest_start = longest_start
        overall_longest_gene = record.id


print ("Longest ORF is in %s starting at %i and is %i bp long." % (overall_longest_gene, 
                                                                   overall_longest_start, 
                                                                   overall_longest_orf))

Longest ORF is in gi|142022655|gb|EQ086233.1|527 starting at 636 and is 1821 bp long.


In [87]:
overall_longest_start = 0
overall_longest_orf = 0
overall_longest_gene = ""

for record in records:
    longest_start, longest_orf = get_longest_for(record)
    if longest_orf>overall_longest_orf:
        overall_longest_orf = longest_orf
        overall_longest_start = longest_start
        overall_longest_gene = record.id


print ("Longest ORF is in %s starting at %i and is %i bp long." % (overall_longest_gene, 
                                                                   overall_longest_start, 
                                                                   overall_longest_orf))

Longest ORF is in gi|142022655|gb|EQ086233.1|45 starting at 385 and is 2394 bp long.


In [88]:
gene_to_check = 'gi|142022655|gb|EQ086233.1|16'

for record in records:
    if record.id == gene_to_check:
        longest_start, longest_orf = get_longest_for(record)
        print ("Longest ORF in %s starting at %i and is %i bp long." % (record.id, 
                                                                        longest_start, 
                                                                        longest_orf))

Longest ORF in gi|142022655|gb|EQ086233.1|16 starting at 1440 and is 1644 bp long.


In [36]:
def splitto(records, length):
    "Splits up records into repeats"
    repeat_list = []
    for record in records:
        repeat_list.extend([record.seq[i:i+length] for i in range(len(record.seq) - length + 1)])
    repeat_dict = {}
    for repeat in repeat_list:
        if repeat in repeat_dict:
            repeat_dict[repeat] += 1
        else:
            repeat_dict[repeat] = 1
    return(repeat_dict)

In [41]:
len_six_dict = splitto(records,6)

most_frequent = max(len_six_dict.values())
print ("Most frequent is %i" % most_frequent)

Most frequent is 153


In [44]:
len_twelve_dict = splitto(records, 12)
most_frequent_twelve = max(len_twelve_dict.values())
twelve_sequences = []
for key in len_twelve_dict.keys():
    if len_twelve_dict[key] == most_frequent_twelve:
        twelve_sequences.append(key)
        
twelve_sequences

[Seq('CATTCGCCATTC'),
 Seq('ATTCGCCATTCG'),
 Seq('TTCGCCATTCGC'),
 Seq('TCGCCATTCGCC')]

In [45]:
len_seven_dict = splitto(records, 7)
most_frequent_seven = max(len_seven_dict.values())
seven_sequences = []
for key in len_seven_dict.keys():
    if len_seven_dict[key] == most_frequent_seven:
        seven_sequences.append(key)
        
seven_sequences

[Seq('CGCGCCG')]

In [35]:
listo

[1, 2, 3, 4, 5, 6]